# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `os` module provides functions to perform tasks such as file and directory operations, process management, and environment variable manipulation.
The `math` module provides a collection of mathematical functions and constants for performing mathematical operations.
* The `geemap` module allows interactive analysis and visualization of GEE datasets in a Jupyter environment.
* The `matplotlib.pyplot` module provides a collection of functions for creating and customizing plots, diagrams and visualizations.
* The `google.colab` module provides access to some of the unique features and functionality of Google Colab.

In [1]:
!pip install geemap
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00


In [2]:
import ee
import os
import math
import geemap
import ffmpeg

import matplotlib.pyplot as plt

from google.colab import drive

***Update the geemap package***

If you encounter errors with this notebook, please remove the comment in the line below to update the geemap package to the latest version from GitHub. Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [3]:
# geemap.update_package()

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=1XBUQzZ9PxfFyhOD4oIERqIwR8TRLDd8SAvNVLxafso&tc=2q1E2Km9E66xLBOgONpq3mvaJeL-tfz43lmG1yt4KFI&cc=yJBD3jMe96YQMG94vmkQOjC28DDp3whvl5RGtJ2Nb_s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkmm5doZCL_xz3l4S-xW55hvqznCQFnWR5lvoXh2cyrFTsZUQBFb9o

Successfully saved authorization token.


## **2.2) GD**

The `drive.mount` function allows access to specific folders of Google Drive. Granting access to Google Drive allows code running in the notebook to modify files in Google Drive.

**Note:** When using the `Mount Drive` button in the file browser, no authentication codes are required for notebooks edited only by the current user.

In [7]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# **3) Functions**

Data Processing

In [8]:
def power_to_db(raster):
  """
  Description:
    Converts pixel values from power scale to dB scale.

  Arguments:
    raster (ee.Image): The raster with pixel values in power scale.

  Returns:
    The raster with pixel values in dB scale.
  """
  return ee.Image(10).multiply(raster.log10())


def db_to_power(raster):
  """
  Description:
    Converts pixel values from dB scale to power scale.

  Args:
    raster (ee.Image): The raster with pixel values in dB scale.

  Returns:
    The raster with pixel values in power scale.
  """
  return ee.Image(10).pow(raster.divide(10))


def refined_lee(raster):
  """
  Description:
    Applies the refined Lee speckle filter to an raster.

  Arguments:
    raster (ee.Image): The raster to apply the filter on.

  Returns:
    The filtered raster layer.
  """
  def computations(b):
    img = raster.select([b])

    # img must be in natural units, i.e. not in dB!
    # Set up 3x3 kernels
    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)

    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])

    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # and add the bands for rotated kernels
    for i in range(1, 4):
      dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
      dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
      dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
      dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

    # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    # A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var)

    return dir_mean.add(b.multiply(img.subtract(dir_mean))) \
      .arrayProject([0])                                    \
      .arrayFlatten([["sum"]])                              \
      .float()

  bandNames = raster.bandNames()
  raster = db_to_power(raster)

  result = ee.ImageCollection(bandNames.map(computations)).toBands().rename(bandNames)
  return power_to_db(ee.Image(result))


def slope_correction(raster, dem):
  """
  Description:
    Performs slope correction on the input raster using a digital elevation model (DEM).

  Arguments:
    raster (ee.Image): The raster to perform slope correction on.
    dem (ee.Image): The digital elevation model used for correction.

  Returns:
    The slope-corrected raster layer.
  """
  # Obtain the geometry of the image.
  imgGeom = raster.geometry()
  # Clip the extent of the dem layer.
  srtm = ee.Image(dem).clip(imgGeom)
  # Convert pixel values from dB scale to linear scale.
  sigma0Pow = db_to_power(raster)

  # 2.1.1 Radar geometry
  # Compute the mean aspect of the terrain over the region covered by the raster.
  theta_i = raster.select("angle")
  phi_i = ee.Terrain.aspect(theta_i)                                        \
    .reduceRegion(ee.Reducer.mean(), theta_i.get("system:footprint"), 1000) \
    .get("aspect")

  # 2.1.2 Terrain geometry
  # Calculate the slope and aspect of the terrain over the region covered by the raster.
  alpha_s = ee.Terrain.slope(srtm).select("slope")
  phi_s = ee.Terrain.aspect(srtm).select("aspect")

  # 2.1.3 Model geometry
  # reduce to 3 angle
  phi_r = ee.Image.constant(phi_i).subtract(phi_s)

  # Perform some mathematical conversions
  # (convert pixel values from degrees to radians).
  phi_rRad = phi_r.multiply(math.pi / 180)
  alpha_sRad = alpha_s.multiply(math.pi / 180)
  theta_iRad = theta_i.multiply(math.pi / 180)
  ninetyRad = ee.Image.constant(90).multiply(math.pi / 180)

  # slope steepness in range (eq. 2)
  alpha_r = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

  # slope steepness in azimuth (eq 3)
  alpha_az = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

  # local incidence angle (eq. 4)
  theta_lia = (alpha_az.cos().multiply((theta_iRad.subtract(alpha_r)).cos())).acos()
  theta_liaDeg = theta_lia.multiply(180 / math.pi)
  # 2.2
  # Gamma_nought_flat
  gamma0 = sigma0Pow.divide(theta_iRad.cos())
  gamma0dB = ee.Image.constant(10).multiply(gamma0.log10())
  ratio_1 = gamma0dB.select("VV").subtract(gamma0dB.select("VH"))

  # Volumetric Model
  nominator = (ninetyRad.subtract(theta_iRad).add(alpha_r)).tan()
  denominator = (ninetyRad.subtract(theta_iRad)).tan()
  volModel = (nominator.divide(denominator)).abs()

  # apply model
  gamma0_Volume = gamma0.divide(volModel)
  gamma0_VolumeDB = ee.Image.constant(10).multiply(gamma0_Volume.log10())

  # we add a layover/shadow maskto the original implmentation
  # layover, where slope > radar viewing angle
  alpha_rDeg = alpha_r.multiply(180 / math.pi)
  layover = alpha_rDeg.lt(theta_i)

  # shadow where LIA > 90
  shadow = theta_liaDeg.lt(85)

  # calculate the ratio for RGB vis
  ratio = gamma0_VolumeDB.select("VV").subtract(gamma0_VolumeDB.select("VH"))

  output = gamma0_VolumeDB.addBands(ratio).addBands(alpha_r).addBands(phi_s).addBands(theta_iRad) \
    .addBands(layover).addBands(shadow).addBands(gamma0dB).addBands(ratio_1)

  return raster.addBands(
    output.select(["VV", "VH"], ["VV", "VH"]),
    None,
    True
  )

Data Visualization

In [9]:
def calculate_histogram(image, bandName, region):
  """
  Description:
    Calculates the histogram for a specified band of an Earth Engine image within a given region.

  Arguments:
      image (ee.Image): The Earth Engine image for which to calculate the histogram.
      bandName (str): The name of the band for which to compute the histogram.
      region (ee.Geometry): The region of interest (ROI) where the histogram is computed.

  Returns:
      A GEE dictionary containing the histogram values for the specified band within the given region.
  """
  return image.select(bandName).reduceRegion(**{
    "reducer": ee.Reducer.autoHistogram(),
    "geometry": region,
    "scale": 10,
    "maxPixels": 1e9
  }).get(bandName)


def plot_histograms(histograms, plotTitle, destinationPath):
  """
  Description:
    Creates a histogram plot from a dictionary and saves the plot to a file. The dictionary
    is a list of tuples, where each tuple contains two elements: a value and
    its corresponding frequency in the histogram.

  Arguments:
  histograms (dict): A dictionary containing histogram data for each identifier.
  destinationPath (str): The path where the plot will be saved.

  Returns:
    None
  """
  hexValues = {"pre_event": "#deb887", "post_event": "#45b6fe"}

  # Create a figure and a set of subplots
  fig, ax = plt.subplots(figsize=(10, 8))

  # Set plot title
  plt.title(plotTitle, fontsize=18, fontweight="bold", color="#595959")

  # Edit axes

  plt.xticks(fontsize=14, fontweight="bold", color="#595959")
  plt.yticks(fontsize=14, fontweight="bold", color="#595959")

  # Add grid lines
  plt.grid(True, linestyle="--", color="white")

  # Set facecolor
  ax.set_facecolor("lightgray")

  # Set limits
  plt.xlim(-30, 10)

  # Create bar plots
  for key, value in histograms.items():
    x = [item[0] for item in value]
    y = [item[1] for item in value]
    plt.bar(x, y, alpha=0.5, width=0.5, edgecolor="white", color=hexValues[key], label=key)

  # Set legend
  legend = plt.legend()

  for text in legend.get_texts():
    text.set_fontweight("bold")

  # Save plot
  plt.savefig(destinationPath, dpi=500, bbox_inches="tight")
  plt.close()

# **4) Parameters**

In [23]:
# `Digital Elevation Model`
demProvider = "USGS"

# `Sentinel-1 GRD`
b1 = "VV"
b2 = "VH"

chartIdentifiers = {
  "pre_event": "003397_003F4F_E129",
  "post_event": "004447_005723_9D4B"
}

rasterIdentifiers = {
  "pre_event": "S1A_IW_GRDH_1SDV_20141122T163134_20141122T163159_003397_003F4F_E129",
  "post_event": "S1A_IW_GRDH_1SDV_20150202T163132_20150202T163157_004447_005723_9D4B"
}

# Projection of interest.
projectionCRS = "EPSG:4326"
projectionScale = 10

# Runtime stuff.
bufferDistance = -10000

coordinates = [
  20.058040632927096, 38.102793131409605,
  22.899790328988313, 38.49643039052389,
  22.90055614340465,  38.512305042085146,
  22.637037745648232, 39.82427938096713,
  22.59494267298734,  39.989073630192436,
  22.59136250228799,  39.998627426959494,
  19.688106510271055, 39.606279267839795
]

undesiredAreas = [
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/assets/filters/emsr117/flood_surfaces")
]

# GD paths.
destinationFolder = "/content/gdrive/MyDrive/t-h-e-s-i-s/results/emsr117/"

# **5) Configuration**

In [24]:
# `Digital Elevation Models`
demConfigs = {
  "CGIAR": {    # `SRTM Digital Elevation Data Version 4`
    "name": "CGIAR/SRTM90_V4",
    "resolution": 90
  },
  "NASA": {     # `NASA NASADEM Digital Elevation`
    "name": "NASA/NASADEM_HGT/001",
    "resolution": 30
  },
  "USGS": {     # `NASA SRTM Digital Elevation`
    "name": "USGS/SRTMGL1_003",
    "resolution": 30
  },
  "ASTER": {    # `AG100: ASTER Global Emissivity Dataset 100-meter V003`
    "name": "NASA/ASTER_GED/AG100_003"
  }
}

# `Sentinel-1 GRD`
s1Config = {
  "name": "COPERNICUS/S1_GRD",
  "mode": "IW",
  "resolution": 10,
  "bands": ["VV", "VH"]
}

sarBluishVisualization = {
  "min": [-18, -25, 1],
  "max": [0, -5, 12],
  "bands": ["VV", "VH", "VVVHD"]
}

sarReddishVisualization = {
  "min": [-25, -25, 0],
  "max": [0, 0, 2],
  "bands": ["VV", "VH", "VVVHQ"]
}

bandCombinations = {
  "sum": {
    "expression": "b1 + b2",
    "name": "b1b2S"
  },
  "difference": {
    "expression": "b1 - b2",
    "name": "b1b2D"
  },
  "product": {
    "expression": "b1 * b2",
    "name": "b1b2P"
  },
  "quotient": {
    "expression": "b1 / b2",
    "name": "b1b2Q"
  },
  "ndpi": {
    "expression": "(b1 - b2) / (b1 + b2)",
    "name": "NDPI"
  }
}

In [25]:
# Retrieve configurations.
demConfig = demConfigs[demProvider]

# Retrieve GEE assets.
areaOfInterest = ee.Geometry.Polygon(coordinates).buffer(bufferDistance, 1)
ring = ee.Geometry.LinearRing(areaOfInterest.coordinates().flatten())
undesiredAreas = ee.FeatureCollection(undesiredAreas).flatten()
borderVectors = ee.FeatureCollection("FAO/GAUL/2015/level2")

# **6) Data Processing**

Define projection of interest.

In [26]:
projection = ee.Projection(projectionCRS).atScale(projectionScale)

Load, filter and process raster collections.

In [27]:
# `Digital Elevation`
elevation = ee.Image(demConfig["name"]).select("elevation").unmask()

# `Sentinel-1 GRD`
preEventRaster = ee.Image("/".join([s1Config["name"], rasterIdentifiers["pre_event"]]))
postEventRaster = ee.Image("/".join([s1Config["name"], rasterIdentifiers["post_event"]]))

# Apply angular-based radiometric slope correction.
preEventRaster = slope_correction(preEventRaster, elevation)
postEventRaster = slope_correction(postEventRaster, elevation)

# Apply a Refined-Lee speckle filter to reduce noise.
preEventRaster = ee.Image(refined_lee(preEventRaster).copyProperties(preEventRaster))
postEventRaster = ee.Image(refined_lee(postEventRaster).copyProperties(postEventRaster))

Handle geometries.

In [28]:
preEventRaster = preEventRaster.clip(areaOfInterest)
postEventRaster = postEventRaster.clip(areaOfInterest)

region = areaOfInterest.difference(undesiredAreas.geometry(), 1)

Transform vector collections into raster representations.

In [29]:
undesiredSurfaces = ee.ImageCollection([
  ee.Image.constant(0).toInt().clip(areaOfInterest),
  ee.Image.constant(1).toInt().clipToCollection(borderVectors),
  ee.Image.constant(0).toInt().clipToCollection(undesiredAreas),
])                                                \
.mosaic()                                         \
.clip(areaOfInterest)                             \
.reproject(projection)

Engineer additional raster bands.

In [30]:
differenceExpression = bandCombinations["difference"]["expression"]
quotientExpression = bandCombinations["quotient"]["expression"]
ndpiExpression = bandCombinations["ndpi"]["expression"]

differenceName = bandCombinations["difference"]["name"]
quotientName = bandCombinations["quotient"]["name"]

In [31]:
# `PRE_VVVHD`
prevvprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b2))

# `POST_VVVHD`
postvvpostvhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b2))

# `VVVVD`
postvvprevvDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b1))

# `VHVHD`
postvhprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b2),
      "b2": preEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b2).replace("b2", b2))

# `VVVHD`
postvvprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b2))

# `PRE_VVVHQ`
prevvprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b2))

# `POST_VVVHQ`
postvvpostvhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b2))

# `VVVVQ`
postvvprevvQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b1))

# `VHVHQ`
postvhprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b2),
      "b2": preEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b2).replace("b2", b2))

# `VVVHQ`
postvvprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b2))

# `PRE_NDPI`
preNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename("PRE_NDPI")

# `POST_NDPI`
postNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  }).rename("POST_NDPI")

# `NDPID`
ndpiDifference = postNDPI.subtract(preNDPI).rename("NDPID")

# Incorporate extra bands into both pre- & post- event rasters.
preEventRaster = preEventRaster.addBands([prevvprevhDifference, prevvprevhQuotient])
postEventRaster = postEventRaster.addBands([postvvpostvhDifference, postvvpostvhQuotient])

# **7) Data Visualization**

Create histograms to evaluate the distribution of band values.

In [32]:
vvRaster = ee.ImageCollection([preEventRaster.select(b1), postEventRaster.select(b1)]).toBands()  \
  .rename([chartIdentifiers["pre_event"], chartIdentifiers["post_event"]])                        \
  .updateMask(undesiredSurfaces)

vhRaster = ee.ImageCollection([preEventRaster.select(b2), postEventRaster.select(b2)]).toBands()  \
  .rename([chartIdentifiers["pre_event"], chartIdentifiers["post_event"]])                        \
  .updateMask(undesiredSurfaces)

# Calculate total pixel count.
pixelsCount = ee.Image.constant(1).clip(areaOfInterest)   \
  .setDefaultProjection(projection)                       \
  .reduceRegion(**{
    "reducer": ee.Reducer.count(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  })

# Calculate mean values.
prevvMean = vvRaster.select(chartIdentifiers["pre_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["pre_event"])

prevhMean = vhRaster.select(chartIdentifiers["pre_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["pre_event"])

postvvMean = vvRaster.select(chartIdentifiers["post_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["post_event"])

postvhMean = vhRaster.select(chartIdentifiers["post_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["post_event"])

# Create histograms.
bands = list(chartIdentifiers.values())

vvHistograms = ee.Dictionary.fromLists(bands, [calculate_histogram(vvRaster, band, areaOfInterest) for band in bands])
vhHistograms = ee.Dictionary.fromLists(bands, [calculate_histogram(vhRaster, band, areaOfInterest) for band in bands])

Rename histogram keys

In [33]:
vvHistograms = vvHistograms.rename(list(chartIdentifiers.values()), list(chartIdentifiers.keys())).getInfo()

In [ ]:
vhHistograms = vhHistograms.rename(list(chartIdentifiers.values()), list(chartIdentifiers.keys())).getInfo()

In [22]:
# Construct output chart paths.
plotIdentifier = "_".join(list(chartIdentifiers.values()))
vvDestinationPath = os.path.join(destinationFolder, "histograms", f"{plotIdentifier}_VV")
vhDestinationPath = os.path.join(destinationFolder, "histograms", f"{plotIdentifier}_VH")

# Plot histograms.
plot_histograms(vvHistograms, "VV Histograms", vvDestinationPath)
plot_histograms(vhHistograms, "VH Histograms", vhDestinationPath)

# **8) Console**

In [ ]:
print(pixelsCount)

In [ ]:
print("*Mean values*", {
  "pre VV": prevvMean.getInfo(),
  "post VV": postvvMean.getInfo(),
  "pre VH": prevhMean.getInfo(),
  "post VH": postvhMean.getInfo()
})

Acquisition Parameters Comparison

In [ ]:
print(preEventRaster.toDictionary().getInfo())
print(postEventRaster.toDictionary().getInfo())

# **9) Map Visualization**

In [ ]:
Map = geemap.Map()

Map.centerObject(areaOfInterest)

Map.addLayer(preEventRaster, sarReddishVisualization, "rasters: S1-GRD (pre-red)")
Map.addLayer(postEventRaster, sarReddishVisualization, "rasters: S1-GRD (post-red)")

Map.addLayer(preEventRaster, sarBluishVisualization, "rasters: S1-GRD (pre-blue)")
Map.addLayer(postEventRaster, sarBluishVisualization, "rasters: S1-GRD (post-blue)")

Map.addLayer(postvvprevvDifference, {min: -10, max: 10}, "rasters: VVVVD")
Map.addLayer(postvhprevhDifference, {min: -10, max: 10}, "rasters: VHVHD")
Map.addLayer(postvvprevhDifference, {min: -8, max: 12}, "rasters: VVVHD")

Map.addLayer(postvvprevvQuotient, {min: 0, max: 3.5}, "rasters: VVVVQ")
Map.addLayer(postvhprevhQuotient, {min: 0, max: 3.5}, "rasters: VHVHQ")
Map.addLayer(postvvprevhQuotient, {min: 0, max: 3.5}, "rasters: VVVHQ")

Map.addLayer(ndpiDifference, {min: -1, max: 1}, "NDPID")
Map.addLayer(postNDPI, {min: -1, max: 1}, "POST_NDPI")
Map.addLayer(preNDPI, {min: -1, max: 1}, "PRE_NDPI")

Map.addLayer(ring, {"color": "white"}, "vectors: area")

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

-End of Notebook-